## Prompt Exploratory Data Analysis
Using this notebook, we can analyze and extract key insight from the promt used for trainig our LLM model.

#### Eda Overview
Token Wise:
- Average Tokens Lengt
- Token Lengths Distribution
- Min/Max in tokes 

Coverage:
- Promt Coverage as a funtion of tokens length
- Promt coverage as a funtion of hardcoded LLM context size

In [1]:
# Setup Dependencies
! pip install seaborn matplotlib datasets transformers wordcloud

   ---------------------------------------- 0.0/300.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.2 kB ? eta -:--:--
   ----- --------------------------------- 41.0/300.2 kB 279.3 kB/s eta 0:00:01
   --------- ----------------------------- 71.7/300.2 kB 326.8 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/300.2 kB 423.5 kB/s eta 0:00:01
   ------------------------ ------------- 194.6/300.2 kB 535.8 kB/s eta 0:00:01
   ---------------------------- --------- 225.3/300.2 kB 550.0 kB/s eta 0:00:01
   ----------------------------------- -- 276.5/300.2 kB 472.9 kB/s eta 0:00:01
   -------------------------------------- 300.2/300.2 kB 475.3 kB/s eta 0:00:00


In [1]:
# Load tokenizer
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenizer.eos_token = "EOS"
tokenizer.pad_token = "UNK"
print(f"Loaded tokenizer : {tokenizer}" if tokenizer else "Failed to load")

c:\Users\agusa\anaconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 349kB/s]
c:\Users\agusa\anaconda3\envs\deeplearning\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\agusa\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_

Loaded tokenizer : BertTokenizerFast(name_or_path='sentence-transformers/all-MiniLM-L6-v2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': 'EOS', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': 'UNK', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [2]:
# Prepare training data parser
from enum import Enum 
from pathlib import Path
from datasets import load_dataset
from dataclasses import dataclass, field

class Scope(Enum):
    TRAIN = "train"
    
@dataclass
class DataSample:
    about_me: str = field(repr=False)
    context: str = ""
    response: str = ""
    
    def formatted(self):
        """"Train containing raw text fields"""
        template = """
        {ABOUT}
        {CONTEXT}
        {RESPONSE}
        """
        complete = template.format(ABOUT=self.about_me, CONTEXT=self.context, RESPONSE=self.response)
        return complete

class DatasetsGenerator:
    def __init__(self, dataset_file_path: Path, scope: Scope):
        self._scope = scope
        self.tokenizer = tokenizer
        self.dataser = self._load_dataset(dataset_file_path)
        
    @property
    def size(self):
        return len(self.dataset[self._scope.value])  
    
    def _load_dataset(self, dataset_file_path: Path):
        try:
            dataset = load_dataset("json", data_files=str(dataset_file_path))
            print("=", * 10 + " info " + "=" * 10)
            print(f"Dataset Path : {dataset_file_path}")
            print(f"Loaded : {len(dataset[self._scope.value])}") 
        except FileNotFoundError:
            print("JSON file not found.")
        except SyntaxError:
            print(f"JOSN file badly formateed.")
            raise FileNotFoundError("Forgot how to write json")
        
    def tokenized_sample(self, tokenizer):
        formatted_samples = list(map(lambda en: DataSample(**en), self.dataset[self._scope.value]))
        tokenized_samples = []
        for sample in formatted_samples:
            as_str = sample.formatted()
            tokenized = tokenizer(as_str)
            tokenized = tokenized["input_ids"] 
            tokenized_samples.append(tokenized)
        return tokenized_samples
    
    def samples_gen(self):
        formatted_samples = list(map(lambda en: DataSample(**en), self.dataset[self._scope.value]))
        for sample in formatted_samples:
            yield sample              

In [3]:
# 1. Prerequisites
DATASET_ROOT = Path("../modules/q_and_a_dataset_generator/data")
dataset_json_file = DATASET_ROOT / "training_data.json"

# 2. Prepare dataset
dataset = DatasetsGenerator(dataset_file_path=dataset_json_file, scope=Scope.TRAIN)
tokenized_samples = dataset.tokenized_samples(tokenizer)

JSON file not found.


AttributeError: 'DatasetsGenerator' object has no attribute 'tokenized_samples'

In [5]:
dataset_json_file

WindowsPath('../modules/q_and_a_dataset_generator/data/training_data.json')